___
___
# **Importando imagenes GOES-16**
**Google Earth Engine**


---
___


En este Notebook, se exploran diferentes algoritmos de detección de ceniza utilizando imágenes capturadas por el sensor multiespectral ABI de la red satelital GOES-16.

Utilizamos el evento de paroxismo del volcán Popocatépetl como objeto de estudio. Este código permite exportar imágenes y *time-lapses* de una ventana temporal de nuestra elección.

\


___
___
## **Creando el ambiente de trabajo**
___
___

### **Autenticación**

El primer paso es autorizar este Notebook para poder acceder a la base de datos de Google Earth Engine (GEE). Este bloque de código nos arrojará un link que nos rediccionará a la página de GEE, donde necesitamos iniciar sesión y autorizar los permisos que pide. Al final nos dará un token de acceso que copiaremos en el recuadro de input.

In [ ]:
import ee

ee.Authenticate()
ee.Initialize(project='san-pinon') # Usar tu project ID

Authenticating using Notebook auth...
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=Fc_8E3CKmWm97k04L9Ic15XhUQvixngMB21CJwpoCcY&tc=gRg10j5hrgGwkRyG_2Kb0zYwMgO5jtowY8lUykokiAA&cc=QmgOo2UWc2XaLdafuNWfgPdjq082e7nJbmKY_gk8Ppc

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXnNgqbBI9UyYVmavOJXgIXwii191l6mTBbYA8uaEhi9YD9QBxGinnc

Successfully saved authorization token.
✓ Successfully initialized!


### **Instalación de paqueterías extras**

La siguiente librería es necesaria para generar mapas.

In [ ]:
import geemap         # Improved mapping library

___
___

## **Visualización de imágenes**

___
___

### **Data import**

Lo que sigue es seleccionar la colección de datos satelitales de nuestro interés desde la base de datos de GEE, que en este caso serán los del GOES-16 (CONUS) Conus & Moisture Imagery. Estas imágenes tienen resolución de 2Km p/ píxel.

Para saber el código de la colección de imágenes que quiero exportar necesito buscar en la [base de datos](https://developers.google.com/earth-engine/datasets/catalog) de Earth Engine

In [ ]:
# Importa la colección de imagenes satelitales
image_collection = ee.ImageCollection("NOAA/GOES/16/MCMIPC")

Para filtrar esta colección, al ser una imagen muy grande, además de saber las coordenadas de nuestra zona de interés, es importante indicar la fecha y hora en la que queremos extraer imagenes porque esta red satelital geoestacionaria registra una imagen cada 5-15 minutos.
\
\
En nuestro caso, tomaremos la actividad de paroxismo que comenzó el 21 de mayo de 2023, pero debemos dar un rango de hora en el que queremos importar imagenes (usando *.filterDate* en nuestra colección de imágenes). También tomaremos un rango de las 16:30-17:00 hrs UTC y le diremos que tome la primer imagen.


Para filtrar la zona que queremos estudiar usamos *.filterBounds* con la herramienta *ee.Geometry.Point*, pero hay que tener **cuidado** porque tiene los índices invertidos; en vez de ser lat-long, es long-lat.


Como no especificamos qué bandas quiero, el código importa todas las que tiene el ABI.


Entonces para generar esta imagen, la mando buscar en la *image_collection* que contiene las imágenes satelitales que seleccioné.


In [ ]:
# lat, long
lat = 19.022685
lng = -98.623038
# Coords copiadas de Google
coords_latlong = [lat, lng]          # [lat, long]
# Coords en formato long-lat
coords_longlat = list(reversed(coords_latlong))   # [long, lat]


# Filtrado de image_collections para extraer la imagen de interés
imagen = ee.Image(image_collection
                 .filterDate('2023-05-21T18:30:00', '2023-05-21T19:00:00')
                            # Tiempos en UTC (México tiene UTC-6hrs)
                 .filterBounds(ee.Geometry.Point(coords_longlat))
                 .first()) # Primer imagen de la lista

### **Image pre-processing**

Al ser un satélite geoestacionario, el ángulo azimutal cambia todo el tiempo, por lo que debemos normalizar las bandas usando este ángulo. Tambien se le aplica un *offset* a la nueva imagen escalada para que no haya distorción.

\

También es necesario generar una banda *GREEN* porque el sensor ABI sólo registra *RED* y *BLUE*. Nos ayudaremos de una banda en el NIR para generar un verde sintético.

In [ ]:
NUM_BANDS = 33
BLUE_BAND_INDEX = (1 - 1) * 2
RED_BAND_INDEX = (2 - 1) * 2
NIR_BAND_INDEX = (3 - 1) * 2
GREEN_BAND_INDEX = NUM_BANDS - 1

def applyScaleAndOffset(img):

  bands = [0] * NUM_BANDS

  names = img.select('CMI_C..').bandNames()
  for i in range (1, 17):
    num = 100+i
    bandName = 'CMI_C' + str(num)[1:3]
    offset = ee.Number(img.get(bandName + '_offset'))
    scale =  ee.Number(img.get(bandName + '_scale'))
    bands[(i-1) * 2] = img.select(bandName).multiply(scale).add(offset)

    dqfName = 'DQF_C' + str(num)[1:3]
    bands[(i-1) * 2 + 1] = img.select(dqfName)


  green = img.expression(
        ' 0.45 * red + 0.10 * nir + 0.45 * blue ', {
        'nir' : bands[NIR_BAND_INDEX],
        'red' : bands[RED_BAND_INDEX],
        'blue': bands[BLUE_BAND_INDEX]
        }
        ).rename('GREEN')
  bands[GREEN_BAND_INDEX] = green


  return ee.Image(ee.Image(bands).copyProperties(img, img.propertyNames()))

image = applyScaleAndOffset(imagen)

### **Generando el mapa**

Ahora, para generar el mapa interactivo, debemos introducir las coordenadas en las que queremos que esté centrado, donde tambíen necesitamos coordenadas *long-lat*.

In [ ]:
# Creamos el mapa
map1 = geemap.Map()
# Configuramos dónde estará centrado
map1.set_center(lng, lat, 8) #long, lat, zoom

Ahora debemos mandar llamar la imagen que importamos para que también se vea en el mapa.

Antes de esto, debemos delimitar las bandas (y su rango dinámico) que queremos importar en el mapa para poder verlas, en este caso serán las bandas RGB y les asignamos un rango arbitrario.
\
\
GOES es un caso particular porque no existe una banda *verde* como tal , por lo que debemos simularla con BLUE, RED y NIR. [[1](https://doi.org/10.1029/2018EA000379)]

In [ ]:
# Defino las bandas correspondientes a RGB para tener una imagen "true color"
BLUE = 'CMI_C01'
RED  = 'CMI_C02'
NIR  = 'CMI_C03'
GREEN = 'GREEN'


# Para conseguir true-color, se tiene que acomodar en el orden R,G,B
rgb_params = {
    'bands': [RED, GREEN, BLUE],
    'min': 0,
    'max': 0.6,
    'gamma' : 1.3,
}

# Los parámetros 'min' y 'max' filtran los valores que puede tomar
# cada subpixel en la imagen respecto a un máximo (gamma)

Finalmente, añadimos al mapa la imagen con los parámetros que acabamos de definir y lo mandamos llamar:

In [ ]:
shapef = ee.FeatureCollection("FAO/GAUL/2015/level1")
shapef = shapef.style(
    color = 'white',
    width = 1.5,
    fillColor = 'ff475700'  # with alpha set for partial transparency
)

In [ ]:
# Añade la capa con la imagen RGB (se puede cambiar la opacidad desde el mapa)
map1.add_layer(image, rgb_params, 'Imagen RGB')
map1.addLayer(shapef, None, 'Outline')
# Manda llamar plotear el mapa
map1

Map(center=[19.022685, -98.623038], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Sea…

---
---
## **Ash Cloud Algorithm**
---
---

### **Physics of the problem**

La detección de ceniza volcánica utiliza bandas en el IR con longitudes de onda de unos cuantos $\mu m$. Las diferentes propiedades *microfísicas* de las partículas de ceniza nos ayudan a distinguirlas de otras partículas de agua/hielo/polvo en la atmósfera.
\
\
Existen múltiples algoritmos que nos permiten detectar las nubes de ceniza volcánica, la misma NOAA nos provee de uno muy poderoso que se puede usar con el GOES-16, pero que ha mostrado requerir de un gran poder computacional debido a la complejidad de este.
\
\
En este proyecto, exploraremos la calidad 3 algoritmos relativamente sencillos, en el que se usan 2, 3 y 5 bandas. Conforme más información tomamos del sistema, más seguros podemos estar de que lo que estamos observando es ceniza volcánica.
\
\
Las bandas espectrales (y la λ en la que están centradas) del sensor ABI que utilizaremos son:


* Banda 11 [CMI_C11] (8.5 μm)
* Banda 13 [CMI_C13] (10.3 μm)
* Banda 15 [CMI_C15] (12.3 μm)
* Banda 16 [CMI_C16] (13.2 μm)

El algoritmo consiste principalmente en calcular la diferencia entre temperaturas de brillo a diferentes longitudes de onda. Esta *brightness temperature* tiene unidades de $K$[Kelvin] y nos las dan las bandas en el infrarrojo, como las antes mencionadas.

### **Método de las 2 bandas (Método 1)**

El primer método propuesto para la detección de ceniza volcánica busca la *Brightness Temperature Difference* (BTD) entre las bandas con longitudes de onda ~10 y ~12 μm, correspondiento a las bandas 13 y 15 del sensor ABI.
\
\
La condición que tiene que cumplir el pixel para que se considere que SÍ contine ceniza es:

\begin{align}
  \text{BT}(10) - \text{BT}(12) < 0\text{K}
\end{align}

Es decir, esta diferencia es negativa cuando el pixel contiene ceniza, mientras que es positiva cuando este no tiene (la suficiente).
\
\
Para poder filtrar bien la imagen y sólo ver los pixeles con ceniza, debemos poner esta conidición como una *máscara* en nuestra imagen.
\
La condición es:

In [ ]:
# La siguiente linea [a ? b : c] se lee "if a then b, otherwise c", donde:
#   a := BT11 - BT12 < K
#   b := 1
#   c := 0
# Aquí K depende depende de la humedad de la zona
meth1mask = image.expression(
    "(BT10 - BT12 < 0) ? 1 "
    ": 0", {
        'BT10' : image.select('CMI_C13'),
        'BT12' : image.select('CMI_C15')
    }
)

# Esta ^ máscara le asigna valor 1 a los pixeles que nos interesan
# y 0 a los que no cumplen la condición.


# Para separar los pixeles que nos interesan en otra capa,
# aplicamos .selfMask() a la máscara, es decir:

imgmasked1 = meth1mask.selfMask()
# Aquí los pixeles con vlaor 0 se vuelven transparentes

Ya que tenemos nuestra máscara, debemos aplicarla a nuestro mapa, para que los pixeles que cumplen la condición, se muestren.

In [ ]:
# Creamos el mapa con una capa true color de fondo
map2 = geemap.Map()
map2.set_center(lng, lat, 8)
map2.addLayer(image, rgb_params, 'Imagen RGB')

# Añadimos la capa con la mascara y la resaltamos de rojo
palette = ['red']
map2.addLayer(imgmasked1, {'palette': palette}, 'Masked image Meth1')
map2.addLayer(shapef, None, 'Outline')
map2

Map(center=[19.022685, -98.623038], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Sea…

### **Método de las 3 bandas (Método 2)**

El Método 1 tiene limitaciones, existen la sobre-estimaciones y las sub-estimaciones. Dentro de las sub-estimaciones, en el caso de regiones tropicales (lat = $\pm 20°$) como en donde se encuentra el Popocatépetl, la columna de vapor de agua encima o debajo de la nube de ceniza desplaza la BTD hacia los positivos; en los climas fríos también pueden generar sub-estimaciones, desplazando la diferencia BTD a los positivos. La sobreestimación ocurre en zonas áridas y cuando hay muchas presencia de minerales en el aire. En nuestro caso, se debe a una posible sub-estimación, por lo que debe contemplar que hay pixeles que no están.

\

Entonces, el siguiente paso para optimizar el código es considerar una tercera banda espectrar, que remueve los *artifacts*, i.e. pixeles con detección falso-positivo o falso-negativo. Esto se consigue con la BTD entre las longitudes de $11\mu m$ y $8.5 \mu m$, en donde los pixeles, no sólo deben cumplir la condición del Método 1 si no que, también deben cumplir la condición:

\begin{align}
  \text{BT}(8.5) - \text{BT}(11) > 0\text{K}
\end{align}

Por lo que nuestra máscara se ve como:

In [ ]:
# Siguiendo la condición anterior la expresión queda como:
meth2mask = image.expression(
    "((BT10-BT12 < 0) && (BT85-BT10 > 0)) ? 1 "
    ": 0", {
        'BT85' : image.select('CMI_C11'),
        'BT10' : image.select('CMI_C13'),
        'BT12' : image.select('CMI_C15')
    }
)


# Volvemos a aplicar la .selfMask() a la nueva máscara
# para separar los pixeles que cumplen la condición.

imgmasked2 = meth2mask.selfMask()

Ahora creamos un nuevo mapa con esta nueva máscara

In [ ]:
# Creamos el mapa con una capa true color de fondo
map3 = geemap.Map()
map3.set_center(lng, lat, 8)
map3.addLayer(image, rgb_params, 'Imagen RGB')

# Añadimos la capa con la mask
palette = ['red']
map3.addLayer(imgmasked2, {'palette': palette}, 'Selfmask Meth2')
map3.addLayer(shapef, None, 'Outline')

map3

Map(center=[19.022685, -98.623038], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Sea…

#### **Reduce Neighborhood**

Para mejorar el algoritmo, aplicamos condiciones más tenues en la frontera de la nube. Para esto, es necesario utilizar la función [*ee.Image.reduceNeighborhood*](https://developers.google.com/earth-engine/apidocs/ee-image-reduceneighborhood) en donde podemos aplicar una función (reducer) a una región (kernel) que rodea un pixel.

\

* El reducer que utilizaremos se llama [*ee.Reducer.anyNonZero*](https://developers.google.com/earth-engine/apidocs/ee-reducer-anynonzero) y nos arroja el valor 1 cuando cualquier pixel dentro del kernel es no-nulo.

* El kernel que usaremos es [*ee.Kernel.square*](https://developers.google.com/earth-engine/apidocs/ee-kernel-square) y considera una zona con $r$ pixeles/metros de radio sobre un pixel, en nuestro caso queremos un radio de 1pixel/2Kmetros


Una vez tenemos la nueva región extendida, aplicamos *.updateMask()* a la imagen original para sólo aplicar las condiciones tenues a los nuevos pixels

In [ ]:
# Aplicamos .reduceNeighborhood() para conseguir la máscara extendida en 1 pixel
mask2_ext1 = meth2mask.reduceNeighborhood(
            reducer = ee.Reducer.anyNonZero(),
            kernel = ee.Kernel.square(4000, units='meters') # 1pixel = 2000meters
) # Con meters funciona mejor que con pixels ¿why?

# Me recorta la imagen con un pixel más que la mascara2
imgmask2_ext1 = image.updateMask(mask2_ext1)

Ahora, aplicamos condiciones más tenues a todos los pixels de la nueva máscara (los que pasaron la primera prueba, cumplen esta). Podemos diferenciarlos de los otros al aplicar condiciones del tipo:

\begin{align*}
\text{Cond_antigua} < \text{BTD} < \text{Cond_nueva}
\end{align*}

In [ ]:
# Aplica condiciones más tenues sólo a la nueva imagen recortada
meth2mask_ext1 = imgmask2_ext1.expression(
    "((BT10-BT12 < 0) && (BT85-BT11 > 0)) ? 1 " # Primeras condiciones
    ": ((0 < BT10-BT12 < 1) && (0 > BT85-BT11 > -1)) ? 2 " # Cond tenues
    ": 0", {
        'BT85' : imgmask2_ext1.select('CMI_C11'),
        'BT10' : imgmask2_ext1.select('CMI_C13'),
        'BT11' : imgmask2_ext1.select('CMI_C14'),
        'BT12' : imgmask2_ext1.select('CMI_C15')
    }
)

# Esta máscara le dará un valor diferente a cada caso
# 1 p/ condiciones estrictas
# 2 p/ condiciones tenues


# Aplicamos .selfMask() a la máscara para extraer la máscara
imgmasked2_ext1 = meth2mask_ext1.selfMask()

Finalmete, podemos hacer el mapa con todas las capas que nos importan para corroborar que funciona el code

Nota: Quitar la RGB si se quiere ver el contorno de la selfmask del método 2 estricto

In [ ]:
# Creamos el mapa con una capa true color de fondo
map5 = geemap.Map()
map5.set_center(lng, lat, 8)
map5.addLayer(image, rgb_params, 'Imagen RGB')
map5.addLayer(imgmask2_ext1, rgb_params, 'Image Mask') # nuevos pixels (original+contorno)

# Añadimos la capa con la mask
palette = ['red'] # paleta Maks2
palette2 = ['black', 'red', 'orange'] #paleta Mask2 extendida
map5.addLayer(imgmasked2, {'palette': palette}, 'Selfmask Meth2') # Mask2 estricta
map5.addLayer(imgmasked2_ext1, {'palette': palette2}, 'Selfmask Meth2 Extendida') # Mask2 extendida
map5.addLayer(shapef, None, 'Outline')

map5

Map(center=[19.022685, -98.623038], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Sea…

### **RGB modificado**

Considerando las secciones anteriores, podemos construir una nueva imagen RGB que nos permita ver mejor la nube de ceniza. Podemos ayudarnos de la documentación que nos ofrece GOES en el archivo [Ash RGB](https://rammb2.cira.colostate.edu/wp-content/uploads/2020/01/GOES_Ash_RGB-1.pdf) en la que modifica las bandas que le damos como imput a la imagen para resaltar la nube de ceniza

\

Considera algunas de las bandas espectrales que utilizamos en este análisis, tomando:

\begin{align*}
\text{NRED} &= \text{BT12.3}-\text{BT10.3} \\
\text{RGREEN} &= \text{BT11.2}-\text{BT8.4} \\
\text{NBLUE} &= \text{BT10.3}
\end{align*}

Por lo que los pixeles que cumplan la condición del Método 1, resaltarán en roj. Las bandas NGREEN y NBLUE nos ayudaran a identificar el S02 y ciertas nubes.

\

Entonces, al construir estas nuevas bandas, podemos aplicarlas a la nueva imagen como:

In [ ]:
# Defino las bandas correspondientes a RGB para tener una imagen "true color"
NRED = image.select('CMI_C15').subtract(image.select('CMI_C13')).rename('NRED')
NGREEN  = image.select('CMI_C14').subtract(image.select('CMI_C11')).rename('NGREEN')
NBLUE = image.select('CMI_C13').rename('NBLUE')
new_bands = ee.Image([NRED, NGREEN, NBLUE])
image = image.addBands(new_bands)


# Para conseguir true-color, se tiene que acomodar en el orden R,G,B
rgb_fake = {
    'bands': ['NRED', 'NGREEN', 'NBLUE'],
    'min': [-6.7, -6, 243.6],
    'max': [2.6, 6.3, 302.4],
    'gamma' : [1, 1, 1],
}

# Los parámetros 'min' y 'max' filtran los valores que puede tomar
# cada subpixel en la imagen respecto a un máximo (gamma)

Construimos el mapa con la banda RGB *fake* y le ponemos la última máscara que construimos para poder comparar resultados

In [ ]:
# Creamos el mapa con una capa true color de fondo
map6 = geemap.Map()
map6.set_center(-98.623038, 19.022685, 8)
map6.addLayer(image, rgb_params, 'Imagen RGB')
map6.addLayer(image, rgb_fake, 'Imagen RGB fake')

# Añadimos la capa con la mask
palette = ['red']
map6.addLayer(imgmasked2_ext1, {'palette': palette2}, 'Selfmask Meth2 Extendida') # Mask2 extendida
map6.addLayer(shapef, None, 'Outline')

map6

Map(center=[19.022685, -98.623038], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Sea…

---
---
## **Animación**
---
---

En esta sección aplicaremos el algoritmo a una colección de imágenes que abarcará aproximadamente 4hrs de tiempo real.

\

Construimos nuestra colección como:

In [ ]:
# Filtramos la colección de GOES16 por fecha y hora
img_col = image_collection.filterDate('2023-05-21T14:00:00', '2023-05-21T19:40:00')

### **Funciones sobre una colección**

Para aplicar una función sobre todas las imágenes de la colección, debemos construirlas como funciones de Python del tipo:
```
def function(image):
  new_img = image.algoritmo()
  return new_img
```
Y para aplicarlas, usamos la función *ee.ImageCollection.map(function)*. La primera función (*applyScaleAndOffset*) ya la tenemos, que nos aplica correcciones de escalamiento y de offset.

Por lo que:



In [ ]:
# Aplica la función applyScaleAndOffset a toda la colección
img_col = img_col.map(applyScaleAndOffset)

### **Imágenes RGB**

Ya que sabemos que el *nuevo* RGB nos permite identificar mejor la nube de ceniza volcánica, construiremos 2 colecciones rgb, una normal y la optimizada para poder compararlas. Estas serían de la forma:

In [ ]:
Longitude: -98.45397949218751
Latitude: 19.606368915776006
text_coords = [-98.45397949218751, 19.606368915776006] #[long, lat]
# Función que genera imagen RGB
def IMGRGB(img):
  # Asignamos parámetros de la imagen

  img_rgb = img.visualize(
      bands = [RED, GREEN, BLUE], # Los definimos desde antes
      min = 0,
      max = 0.6,
      gamma = 1.3
  )
  return  img_rgb.blend(shapef)

# Colección RGB normal
rgb_col = img_col.map(IMGRGB)

Y para el RGB modificado tenemos:

In [ ]:
# Función que añade las bandas RGB modificadas a la colección de imágenes
def NewRGB(img):
  NRED = img.select('CMI_C15').subtract(img.select('CMI_C13')).rename('NRED')
  NGREEN  = img.select('CMI_C14').subtract(img.select('CMI_C11')).rename('NGREEN')
  NBLUE = img.select('CMI_C13').rename('NBLUE')
  new_bands = ee.Image([NRED, NGREEN, NBLUE])
  return img.addBands(new_bands)

# Aplicamos la función de las nuevas bandas
newimg_col = img_col.map(NewRGB)


# Generamos la colección de imágenes RGB, con la mismas propiedades de la col. original
def NewRGBCol(img):
  newrgbimg = img.visualize(
      bands = ['NRED', 'NGREEN', 'NBLUE'],
      min = [-6.7, -6, 243.6],
      max = [2.6, 6.3, 302.4],      # Parámetros paraa que resalte lo que queremos
      gamma = [1,1,1]
      )
  return newrgbimg.blend(shapef)

# Aplicamos la función para sólo tener la col. de imgs RGB
newrgb_col = newimg_col.map(NewRGBCol)

### **Imágenes RGB + MASK**

Si queremos conseguir una imagen de la máscara y que en el fondo se vea la imagen RGB, debemos aplicar todas estas condiciones dentro de una misma función para que se pueda hacer una unión **1-1**.

#### **Método 1**

In [ ]:
# Función de RGB_mod + MASK 1
def RGBMASK1(img):
  # Asignamos parámetros a la img rgb modificada
  rgbimg = img.visualize(
      bands = ['NRED', 'NGREEN', 'NBLUE'],
      min = [-6.7, -6, 243.6],
      max = [2.6, 6.3, 302.4],
      gamma = [1,1,1]
      )
  # Conseguimos los pixeles de la máscara
  methmask1 = img.expression(
      "(BT10-BT12 < 0) ? 1 "
      ": 0", {
         'BT10' : img.select('CMI_C13'),
         'BT12' : img.select('CMI_C15')
     }).selfMask()
  # Asignamos parámetros a la máscara
  mask1 = methmask1.visualize(
      palette = ['red'],
      opacity = 0.5
  )
  return rgbimg.blend(mask1).set('system:time_start', img.get('system:time_start')).blend(shapef)

# Aplicamos la función para conseguir la imagen con rgb+msk1
rgbmask1_col = newimg_col.map(RGBMASK1)

#### **Método 2**

In [ ]:
# Función de RGB_mod + MASK 1
def RGBMASK2(img):
  # Asignamos parámetros a la img rgb modificada
  rgbimg = img.visualize(
      bands = ['NRED', 'NGREEN', 'NBLUE'],
      min = [-6.7, -6, 243.6],
      max = [2.6, 6.3, 302.4],
      gamma = [1,1,1]
      )
  # Conseguimos los pixeles de la máscara
  methmask2 = img.expression(
      "((BT10-BT12 < 0) && (BT85-BT10 > 0)) ? 1 "
      ": 0", {
         'BT85' : img.select('CMI_C11'),
         'BT10' : img.select('CMI_C13'),
         'BT12' : img.select('CMI_C15')
     }).selfMask()
  # Asignamos parámetros a la máscara
  mask2 = methmask2.visualize(
      palette = ['red'],
      opacity = 0.5
  )
  return rgbimg.blend(mask2).set('system:time_start', img.get('system:time_start')).blend(shapef)

# Aplicamos la función para conseguir la imagen con rgb+msk1
rgbmask2_col = newimg_col.map(RGBMASK2)

#### **Método 2 modificado**

In [ ]:
# Función de RGB_mod + MASK 1
def RGBMASK2M(img):
  # Asignamos parámetros a la img rgb modificada
  rgbimg = img.visualize(
      bands = ['NRED', 'NGREEN', 'NBLUE'],
      min = [-6.7, -6, 243.6],
      max = [2.6, 6.3, 302.4],
      gamma = [1,1,1]
      )
  # Conseguimos la máscara
  methmask2 = img.expression(
      "((BT10-BT12 < 0) && (BT85-BT10 > 0)) ? 1 "
      ": 0", {
         'BT85' : img.select('CMI_C11'),
         'BT10' : img.select('CMI_C13'),
         'BT12' : img.select('CMI_C15')
     })
  # Expandimos la máscara
  mask2_ext = methmask2.reduceNeighborhood(
      reducer = ee.Reducer.anyNonZero(),
      kernel = ee.Kernel.square(4000, units='meters') # 1pixel = 2000meters
  )
  # Recortamos la imagen
  img_ext = img.updateMask(mask2_ext)
  # Conseguimos los pixeles de la nueva máscara
  methmask2m = img_ext.expression(
      "((BT10-BT12 < 0) && (BT85-BT10 > 0)) ? 1 " # Cond estrictas
      ": ((0 < BT10-BT12 < 1) && (0 > BT85-BT10 > -1)) ? 2 " # Cond tenues
      ": 0", {
          'BT85' : img_ext.select('CMI_C11'),
          'BT10' : img_ext.select('CMI_C13'),
          'BT12' : img_ext.select('CMI_C15')
      }).selfMask()
  # Asignamos parámetros a la máscara
  mask2m = methmask2m.visualize(
      palette = ['black', 'red', 'orange'],
      opacity = 0.5
  )
  return rgbimg.blend(mask2m).set('system:time_start', img.get('system:time_start')).blend(shapef)

# Aplicamos la función para conseguir la imagen con rgb+msk1
rgbmask2m_col = newimg_col.map(RGBMASK2M)

### **Generando el GIF**

Conseguimos coordenadas de las esquinas opuestas que comprenden nuestra *box* del GIF:


* Longitude: -99.98657226562501
Latitude: 20.179723502765153

* Longitude: -96.7236328125
Latitude: 18.312810846425457

\
Longitude: -96.54785156250001
Latitude: 20.179723502765153

* Longitude: -98.80004882812501
Latitude: 18.838714379258032

* Longitude: -96.83898925781251
Latitude: 20.195190636474504

In [ ]:
# Definimos nuestra Area-Of-Interest
aoi = ee.Geometry.BBox(-98.80004882812501, 18.838714379258032,
                       -96.54785156250001, 20.179723502765153)  # (west, south, east, north)

# Le damos parámetros al video
videoArgs = {
  'dimensions': 768,
  'region': aoi,
  'framesPerSecond': 3,
  'crs': 'EPSG:3857',
  }

Mandamos imprimir una URL que nos lleva al video/GIF generado

In [ ]:
def timelist(img):
  #return ee.Date(img).format('H:m', 'America/Mexico_City')
  return ee.Feature(None, {'time': img.date().format('H:m', 'America/Mexico_City')})


times = img_col.map(timelist).aggregate_array('time').getInfo()


In [ ]:
saved_gif = 'goes.gif'
geemap.download_ee_video(newrgb_col, videoArgs, saved_gif)
#geemap.show_image(saved_gif)

Generating URL...
Please wait ...
The GIF image has been saved to: /content/goes.gif


In [ ]:
text = times
out_gif = 'goesrgbm_time.gif'
geemap.add_text_to_gif(
    saved_gif,
    out_gif,
    xy=('3%', '5%'),
    text_sequence=text,
    font_size=30,
    font_color='#ffffff',
)
geemap.show_image(out_gif)

Output()

### **Generando thumbnails**

Aquí, podemos exportar los mapas de la primera sección, donde se vió el efecto de los algoritmos para una sola imagen. Utilizamos la funciones que se aplican a las colecciones de imágenes porque estas nos formatean la imagen con las layers que queremos, lo que facilita el uso de la función *ee.Image.getThumbURL()*.

\

Aplicamos las funciones a nuestra imagen, tomando la misma región de interés que la que usamos en la animación.

In [ ]:
# Generamos nuestras imagenes
rgb_thumb = IMGRGB(image)
newrgb_thumb = NewRGBCol(image)
mask1_thumb = RGBMASK1(image)
mask2_thumb = RGBMASK2(image)
mask2m_thumb = RGBMASK2M(image)

# Convertimos las ee.Image en imágenes PNG
print(rgb_thumb.getThumbURL({'region': aoi, 'scale': 200, 'format': 'png'}))

https://earthengine.googleapis.com/v1/projects/earthengine-legacy/thumbnails/ade104cec2f464609d4ec7467a0e1765-23c70967d4937d233dbf3323dfda8539:getPixels


In [ ]:
projection = rgb_thumb.select(0).projection().getInfo()
projection
#crs = projection['crs']
crs_transform = projection['transform']

In [ ]:
#geemap.ee_export_image(rgb_thumb, filename="rgb.tif", scale=30, region=aoi)
geemap.ee_export_image(
    rgb_thumb,
    filename="rgb.tif",
    crs='EPSG:3857',
    crs_transform=crs_transform,
    region=aoi,
)

Generating URL ...
Please wait ...
Data downloaded to /content/rgb.tif
